# Task 2.5: Neural Network (MLP Classifier)


## Objective 
Implement MLP Classifier with sklearn, experiment with hidden layer sizes, activation functions, and learning rates, and document convergence behavior.

---

## Overview

In this notebook, we implement a **Multi-Layer Perceptron (MLP) Classifier** for predicting Airbnb listing value categories. We will:

1. Train a baseline MLP model
2. Experiment with different hidden layer architectures
3. Test various activation functions
4. Explore different learning rates
5. Analyze convergence behavior
6. Perform hyperparameter tuning with GridSearchCV
7. Evaluate the final model comprehensively

**Note:** ROC-AUC, Precision-Recall curves and confusion matrices will be handled by **Emircan** in **Task 2.16 && Task 2.17** as part of the comprehensive model evaluation phase.

---

## What is a Multi-Layer Perceptron (MLP)?

An **MLP** is a class of feedforward artificial neural network. It consists of:

- **Input Layer:** Receives the feature data (one neuron per feature)
- **Hidden Layer(s):** Performs non-linear transformations using activation functions
- **Output Layer:** Produces the final class predictions

### How does an MLP learn?

1. **Forward Propagation:** Input data flows through the network, layer by layer
2. **Loss Calculation:** The difference between predictions and actual values is computed
3. **Backpropagation:** Gradients are calculated to determine how to adjust weights
4. **Weight Update:** Weights are adjusted using an optimizer (e.g., Adam)
5. **Iteration:** Steps 1-4 repeat until convergence

### Why use MLP for classification?

- Can learn complex, non-linear decision boundaries
- Flexible architecture (adjustable layers and neurons)
- Works well with scaled numerical features
- Handles multi-class classification naturally

---

## 1. Import Libraries and Load Data

### 1.1 Import Required Libraries

We need several libraries for this task:

| Library | Purpose |
|---------|--------|
| `pandas`, `numpy` | Data manipulation and numerical operations |
| `matplotlib`, `seaborn` | Data visualization |
| `MLPClassifier` | Neural network implementation |
| `GridSearchCV` | Hyperparameter tuning |
| `learning_curve` | Analyzing model performance vs training size |
| `classification_report`, `confusion_matrix` | Model evaluation metrics |
| `pickle` | Saving trained models |
| `time` | Tracking training duration |

In [ ]:
# Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, learning_curve, cross_val_score
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score, 
    precision_recall_fscore_support
)
import warnings
import pickle
import time
import json

# Suppress convergence warnings for cleaner output
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('=' * 60)
print('LIBRARIES IMPORTED SUCCESSFULLY')
print('=' * 60)
print('\nKey libraries loaded:')
print('  • pandas, numpy: Data manipulation')
print('  • matplotlib, seaborn: Visualization')
print('  • sklearn.neural_network: MLP implementation')
print('  • sklearn.model_selection: Cross-validation & tuning')
print('  • sklearn.metrics: Evaluation metrics')

### 1.2 Load Preprocessed Data

We load the **scaled** training and testing datasets created in Week 1 (Task 1.6).

#### Why use scaled data for neural networks?

Neural networks are **sensitive to feature scales** because:

1. **Gradient descent optimization:** Large feature values can cause large gradients, leading to unstable training
2. **Weight initialization:** Networks assume input features are roughly in the same range
3. **Activation function saturation:** Extreme values can saturate activation functions (especially sigmoid/tanh)
4. **Equal feature contribution:** Scaling ensures all features contribute proportionally

#### Data files:
- `X_train_scaled.csv`: Training features (StandardScaler applied)
- `X_test_scaled.csv`: Testing features (StandardScaler applied)
- `y_train.csv`: Training target labels
- `y_test.csv`: Testing target labels

In [ ]:
# Load the preprocessed and scaled datasets
print('Loading preprocessed data...\n')

# Load features and targets
X_train = pd.read_csv('../../data/processed/X_train_scaled.csv')
X_test = pd.read_csv('../../data/processed/X_test_scaled.csv')
y_train_raw = pd.read_csv('../../data/processed/y_train.csv').values.ravel()  # Convert to 1D array
y_test_raw = pd.read_csv('../../data/processed/y_test.csv').values.ravel()    # Convert to 1D array

# Encode string labels to numeric (0, 1, 2)
le = LabelEncoder()
y_train = le.fit_transform(y_train_raw)
y_test = le.transform(y_test_raw)

# Display comprehensive dataset information
print('=' * 60)
print('DATASET INFORMATION')
print('=' * 60)
print(f'\n Training Set:')
print(f'   • Features (X_train): {X_train.shape[0]:,} samples × {X_train.shape[1]} features')
print(f'   • Target (y_train): {y_train.shape[0]:,} samples')
print(f'\n Testing Set:')
print(f'   • Features (X_test): {X_test.shape[0]:,} samples × {X_test.shape[1]} features')
print(f'   • Target (y_test): {y_test.shape[0]:,} samples')
print(f'\n Train/Test Split Ratio: {X_train.shape[0]/(X_train.shape[0]+X_test.shape[0])*100:.0f}% / {X_test.shape[0]/(X_train.shape[0]+X_test.shape[0])*100:.0f}%')

# Display class distribution
print(f'\n Target Classes: {np.unique(y_train)}')
print(f' Number of Classes: {len(np.unique(y_train))}')
print(f'\n Class Distribution in Training Set:')
class_counts = pd.Series(y_train).value_counts().sort_index()
for cls, count in class_counts.items():
    percentage = (count / len(y_train)) * 100
    bar = '█' * int(percentage / 2)
    print(f'   Class {cls}: {count:5,} samples ({percentage:5.2f}%) {bar}')

print('\n' + '=' * 60)

---

## 2. Baseline MLP Model

### Why start with a baseline?

A baseline model serves several important purposes:

1. **Performance benchmark:** Establishes a reference point for measuring improvements
2. **Sanity check:** Verifies that data is loaded correctly and the model can learn
3. **Default behavior:** Shows how sklearn's default parameters perform on our data
4. **Comparison basis:** All experiments will be compared against this baseline

### Default MLPClassifier Parameters:

| Parameter | Default Value | Description |
|-----------|---------------|-------------|
| `hidden_layer_sizes` | (100,) | Single hidden layer with 100 neurons |
| `activation` | 'relu' | Rectified Linear Unit activation |
| `solver` | 'adam' | Adam optimizer (adaptive learning rate) |
| `learning_rate_init` | 0.001 | Initial learning rate |
| `max_iter` | 200 | Maximum training iterations |
| `alpha` | 0.0001 | L2 regularization strength |

### What we're tracking:
- **Training time:** Computational cost
- **Number of iterations:** Epochs until convergence
- **Test accuracy:** Generalization performance
- **Per-class metrics:** Precision, recall, F1-score

In [ ]:
# Train baseline MLP model with default parameters
print('=' * 60)
print('TRAINING BASELINE MLP MODEL')
print('=' * 60)
print('\n Configuration: sklearn default parameters')
print('   • hidden_layer_sizes: (100,) - Single layer, 100 neurons')
print('   • activation: relu - Rectified Linear Unit')
print('   • solver: adam - Adaptive Moment Estimation')
print('   • learning_rate_init: 0.001')
print('   • max_iter: 500 (increased from default 200)')
print('\n Training in progress...\n')

# Create and train baseline model
baseline_mlp = MLPClassifier(random_state=42, max_iter=500)

# Track training time
start_time = time.time()
baseline_mlp.fit(X_train, y_train)
training_time = time.time() - start_time

# Make predictions
y_pred_baseline = baseline_mlp.predict(X_test)
baseline_accuracy = accuracy_score(y_test, y_pred_baseline)

# Display results
print('=' * 60)
print('BASELINE MODEL RESULTS')
print('=' * 60)
print(f'\n⏱  Training Time: {training_time:.2f} seconds')
print(f' Iterations to Convergence: {baseline_mlp.n_iter_}')
print(f' Final Training Loss: {baseline_mlp.loss_:.6f}')
print(f'\n Test Accuracy: {baseline_accuracy:.4f} ({baseline_accuracy*100:.2f}%)')
print(f'\n Detailed Classification Report:')
print('-' * 60)
print(classification_report(y_test, y_pred_baseline))
print('=' * 60)

### Understanding the Classification Report

| Metric | Formula | Interpretation |
|--------|---------|----------------|
| **Precision** | TP / (TP + FP) | Of all predicted positives, how many are correct? |
| **Recall** | TP / (TP + FN) | Of all actual positives, how many did we find? |
| **F1-Score** | 2 × (P × R) / (P + R) | Harmonic mean of precision and recall |
| **Support** | - | Number of actual samples in each class |

**Averages:**
- **Macro avg:** Unweighted mean (treats all classes equally)
- **Weighted avg:** Weighted by class support (accounts for imbalance)

---

## 3. Experiment 1: Hidden Layer Sizes

### What are hidden layers?

Hidden layers are the computational core of neural networks. Each hidden layer:

1. **Receives input** from the previous layer (weighted sum)
2. **Applies activation function** (introduces non-linearity)
3. **Passes output** to the next layer

### Architecture Design Principles:

| Architecture Type | Example | Use Case |
|-------------------|---------|----------|
| **Shallow** | (50,) | Simple patterns, fast training |
| **Single wide** | (100,) | Moderate complexity |
| **Funnel** | (100, 50) | Gradual feature compression |
| **Uniform deep** | (100, 100) | Complex patterns |
| **Deep funnel** | (100, 50, 25) | Hierarchical feature learning |

### Trade-offs:

**Too few neurons/layers (Underfitting):**
- Cannot capture complex patterns
- High bias, low variance
- Poor performance on both train and test

**Too many neurons/layers (Overfitting):**
- Memorizes training data
- Low bias, high variance
- Good train performance, poor test performance

### Architectures to Test:

1. **(50,)** - Simple: 50 neurons, 1 layer
2. **(100,)** - Baseline: 100 neurons, 1 layer
3. **(50, 50)** - Moderate: 50+50 neurons, 2 layers
4. **(100, 50)** - Funnel: 100→50 neurons, 2 layers
5. **(100, 100)** - Wide: 100+100 neurons, 2 layers
6. **(100, 50, 25)** - Deep funnel: 100→50→25, 3 layers
7. **(150, 100, 50)** - Large: 150→100→50, 3 layers

In [ ]:
# Define hidden layer configurations to test
hidden_layer_configs = [
    (50,),           # Simple: 1 layer, 50 neurons
    (100,),          # Baseline: 1 layer, 100 neurons
    (50, 50),        # Moderate depth: 2 layers, 50 each
    (100, 50),       # Funnel: 2 layers, decreasing
    (100, 100),      # Wide: 2 layers, 100 each
    (100, 50, 25),   # Deep funnel: 3 layers, decreasing
    (150, 100, 50),  # Large capacity: 3 layers
]

# Store results
hidden_layer_results = []

print('=' * 60)
print('EXPERIMENT 1: HIDDEN LAYER SIZES')
print('=' * 60)
print('\n Testing 7 different neural network architectures')
print('\n Configurations:')
for i, config in enumerate(hidden_layer_configs, 1):
    total_neurons = sum(config)
    print(f'   {i}. {str(config):20s} | {len(config)} layer(s) | {total_neurons} total neurons')
print('\n' + '-' * 60)

# Test each configuration
for idx, config in enumerate(hidden_layer_configs, 1):
    print(f'\n[{idx}/7] Testing architecture: {config}')
    
    # Create and train MLP
    mlp = MLPClassifier(
        hidden_layer_sizes=config,
        random_state=42,
        max_iter=500
    )
    
    start_time = time.time()
    mlp.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    # Evaluate
    y_pred = mlp.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Store results including loss curve for convergence analysis
    hidden_layer_results.append({
        'config': str(config),
        'accuracy': accuracy,
        'training_time': training_time,
        'n_iter': mlp.n_iter_,
        'final_loss': mlp.loss_,
        'loss_curve': mlp.loss_curve_
    })
    
    print(f'       ✓ Accuracy: {accuracy:.4f} | Time: {training_time:.2f}s | Iterations: {mlp.n_iter_} | Loss: {mlp.loss_:.6f}')

# Create summary DataFrame
hidden_layer_df = pd.DataFrame(hidden_layer_results)

print('\n' + '=' * 60)
print('HIDDEN LAYER EXPERIMENT SUMMARY')
print('=' * 60)
print(hidden_layer_df[['config', 'accuracy', 'training_time', 'n_iter', 'final_loss']].to_string(index=False))
print('\n' + '-' * 60)
best_idx = hidden_layer_df['accuracy'].idxmax()
print(f' Best Architecture: {hidden_layer_df.loc[best_idx, "config"]}')
print(f' Best Accuracy: {hidden_layer_df["accuracy"].max():.4f}')
print(f' Improvement over baseline: {(hidden_layer_df["accuracy"].max() - baseline_accuracy)*100:+.2f}%')
print('=' * 60)

---

## 4. Experiment 2: Activation Functions

### What is an activation function?

Activation functions introduce **non-linearity** into neural networks. Without them, stacking multiple layers would be equivalent to a single linear transformation.

### Activation Functions Comparison:

#### 1. ReLU (Rectified Linear Unit) - Default
```
f(x) = max(0, x)
```
- **Output range:** [0, ∞)
- **Pros:** Fast computation, avoids vanishing gradient, sparse activation
- **Cons:** "Dying ReLU" problem (neurons can become permanently inactive)
- **Best for:** Deep networks, general-purpose classification

#### 2. Tanh (Hyperbolic Tangent)
```
f(x) = (e^x - e^-x) / (e^x + e^-x)
```
- **Output range:** [-1, 1] (zero-centered)
- **Pros:** Zero-centered output, stronger gradients than sigmoid
- **Cons:** Vanishing gradient for extreme values
- **Best for:** When zero-centered outputs are beneficial

#### 3. Logistic (Sigmoid)
```
f(x) = 1 / (1 + e^-x)
```
- **Output range:** [0, 1]
- **Pros:** Smooth gradient, probabilistic interpretation
- **Cons:** Vanishing gradient, not zero-centered, slow convergence
- **Best for:** Output layer in binary classification

### Visual Comparison:
```
ReLU:     ___/     (linear for x>0, zero for x<0)
Tanh:     _--‾     (S-curve from -1 to 1)
Sigmoid:  _-‾      (S-curve from 0 to 1)
```

In [ ]:
# Define activation functions to test
activation_functions = ['relu', 'tanh', 'logistic']
activation_results = []

print('=' * 60)
print('EXPERIMENT 2: ACTIVATION FUNCTIONS')
print('=' * 60)
print('\n Testing 3 different activation functions')
print('\n Fixed architecture: (100, 50) to isolate activation effect')
print('\n Activation functions:')
print('   1. ReLU (Rectified Linear Unit) - f(x) = max(0, x)')
print('   2. Tanh (Hyperbolic Tangent) - f(x) = tanh(x)')
print('   3. Logistic (Sigmoid) - f(x) = 1/(1+e^-x)')
print('\n' + '-' * 60)

# Test each activation function
for idx, activation in enumerate(activation_functions, 1):
    print(f'\n[{idx}/3] Testing activation: {activation.upper()}')
    
    mlp = MLPClassifier(
        hidden_layer_sizes=(100, 50),
        activation=activation,
        random_state=42,
        max_iter=500
    )
    
    start_time = time.time()
    mlp.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    y_pred = mlp.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    activation_results.append({
        'activation': activation,
        'accuracy': accuracy,
        'training_time': training_time,
        'n_iter': mlp.n_iter_,
        'final_loss': mlp.loss_,
        'loss_curve': mlp.loss_curve_
    })
    
    print(f'Accuracy: {accuracy:.4f} | Time: {training_time:.2f}s | Iterations: {mlp.n_iter_} | Loss: {mlp.loss_:.6f}')

# Create summary DataFrame
activation_df = pd.DataFrame(activation_results)

print('\n' + '=' * 60)
print('ACTIVATION FUNCTION EXPERIMENT SUMMARY')
print('=' * 60)
print(activation_df[['activation', 'accuracy', 'training_time', 'n_iter', 'final_loss']].to_string(index=False))
print('\n' + '-' * 60)
best_idx = activation_df['accuracy'].idxmax()
print(f' Best Activation: {activation_df.loc[best_idx, "activation"].upper()}')
print(f' Best Accuracy: {activation_df["accuracy"].max():.4f}')
print('=' * 60)

---

## 5. Experiment 3: Learning Rates

### What is the learning rate?

The learning rate (η) controls the **step size** during gradient descent optimization:

```
new_weight = old_weight - η × gradient
```

### Learning Rate Impact:

| Learning Rate | Behavior | Symptoms |
|---------------|----------|----------|
| **Too Low** (0.0001) | Very small weight updates | Slow convergence, may get stuck |
| **Optimal** (0.001-0.01) | Balanced updates | Smooth, efficient convergence |
| **Too High** (0.1+) | Large weight updates | Oscillation, divergence |

### Visual Analogy:

Imagine descending a mountain to find the lowest valley:

- **Low LR:** Taking tiny steps - safe but extremely slow
- **Optimal LR:** Taking confident strides - efficient progress
- **High LR:** Taking huge leaps - might jump over the valley or fall

### Learning Rates to Test:

1. **0.0001** - Very conservative (10× smaller than default)
2. **0.001** - Default (sklearn standard)
3. **0.01** - Aggressive (10× larger than default)
4. **0.1** - Very aggressive (100× larger than default)

**Note:** We use the Adam optimizer which adapts learning rates, but the initial value still significantly impacts training.

In [ ]:
# Define learning rates to test
learning_rates = [0.0001, 0.001, 0.01, 0.1]
learning_rate_results = []

print('=' * 60)
print('EXPERIMENT 3: LEARNING RATES')
print('=' * 60)
print('\n Testing 4 different learning rates')
print('\n Fixed: architecture=(100, 50), activation=relu')
print('\n Learning rates:')
print('   1. 0.0001 - Very conservative (10× smaller than default)')
print('   2. 0.001  - Default (sklearn standard)')
print('   3. 0.01   - Aggressive (10× larger than default)')
print('   4. 0.1    - Very aggressive (100× larger than default)')
print('\n' + '-' * 60)

# Test each learning rate
for idx, lr in enumerate(learning_rates, 1):
    print(f'\n[{idx}/4] Testing learning rate: {lr}')
    
    mlp = MLPClassifier(
        hidden_layer_sizes=(100, 50),
        learning_rate_init=lr,
        random_state=42,
        max_iter=500
    )
    
    start_time = time.time()
    mlp.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    y_pred = mlp.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    learning_rate_results.append({
        'learning_rate': lr,
        'accuracy': accuracy,
        'training_time': training_time,
        'n_iter': mlp.n_iter_,
        'final_loss': mlp.loss_,
        'loss_curve': mlp.loss_curve_
    })
    
    print(f'Accuracy: {accuracy:.4f} | Time: {training_time:.2f}s | Iterations: {mlp.n_iter_} | Loss: {mlp.loss_:.6f}')

# Create summary DataFrame
learning_rate_df = pd.DataFrame(learning_rate_results)

print('\n' + '=' * 60)
print('LEARNING RATE EXPERIMENT SUMMARY')
print('=' * 60)
print(learning_rate_df[['learning_rate', 'accuracy', 'training_time', 'n_iter', 'final_loss']].to_string(index=False))
print('\n' + '-' * 60)
best_idx = learning_rate_df['accuracy'].idxmax()
print(f' Best Learning Rate: {learning_rate_df.loc[best_idx, "learning_rate"]}')
print(f' Best Accuracy: {learning_rate_df["accuracy"].max():.4f}')
print('=' * 60)

---

## 6. Convergence Behavior Analysis

### What is convergence?

**Convergence** is the process where the model's loss (error) decreases and stabilizes during training. A well-converged model shows:

- Steadily decreasing loss
- Stable final loss value
- No erratic oscillations
- Reaches a plateau

### Why analyze convergence?

1. **Diagnose training issues:** Is the model learning properly?
2. **Compare configurations:** Which settings converge faster/better?
3. **Detect problems:**
   - Slow convergence → Learning rate too low
   - Oscillating loss → Learning rate too high
   - Plateauing early → Model stuck in local minimum
   - Diverging loss → Serious training instability

### Reading Loss Curves:

| Pattern | Meaning | Action |
|---------|---------|--------|
| Smooth decrease → plateau | Healthy convergence | Good! |
| Very slow decrease | Learning rate too low | Increase LR |
| Oscillating/zigzag | Learning rate too high | Decrease LR |
| Increasing loss | Training diverging | Much lower LR |
| Early plateau | Stuck in local minimum | Try different init |

In [ ]:
# Create comprehensive convergence visualization
print('=' * 60)
print('CREATING CONVERGENCE ANALYSIS VISUALIZATIONS')
print('=' * 60)
print('\n Generating 4 plots...\n')

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# ================================================================
# PLOT 1: Hidden Layer Sizes Convergence
# ================================================================
ax1 = axes[0, 0]
for result in hidden_layer_results:
    ax1.plot(result['loss_curve'], label=result['config'], linewidth=2, alpha=0.8)
ax1.set_xlabel('Iterations (Epochs)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Training Loss', fontsize=12, fontweight='bold')
ax1.set_title('Convergence: Hidden Layer Architectures', fontsize=14, fontweight='bold')
ax1.legend(fontsize=9, loc='upper right')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(left=0)

# ================================================================
# PLOT 2: Activation Functions Convergence
# ================================================================
ax2 = axes[0, 1]
colors_act = ['#E74C3C', '#27AE60', '#3498DB']
for idx, result in enumerate(activation_results):
    ax2.plot(result['loss_curve'], label=result['activation'].upper(), 
             linewidth=2.5, alpha=0.9, color=colors_act[idx])
ax2.set_xlabel('Iterations (Epochs)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Training Loss', fontsize=12, fontweight='bold')
ax2.set_title('Convergence: Activation Functions', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11, loc='upper right')
ax2.grid(True, alpha=0.3)
ax2.set_xlim(left=0)

# ================================================================
# PLOT 3: Learning Rates Convergence
# ================================================================
ax3 = axes[1, 0]
colors_lr = ['#9B59B6', '#E67E22', '#1ABC9C', '#E74C3C']
for idx, result in enumerate(learning_rate_results):
    ax3.plot(result['loss_curve'], label=f'LR = {result["learning_rate"]}', 
             linewidth=2.5, alpha=0.9, color=colors_lr[idx])
ax3.set_xlabel('Iterations (Epochs)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Training Loss', fontsize=12, fontweight='bold')
ax3.set_title('Convergence: Learning Rates', fontsize=14, fontweight='bold')
ax3.legend(fontsize=11, loc='upper right')
ax3.grid(True, alpha=0.3)
ax3.set_xlim(left=0)

# ================================================================
# PLOT 4: Best Accuracy Comparison
# ================================================================
ax4 = axes[1, 1]
experiments = ['Hidden\nLayers', 'Activation\nFunctions', 'Learning\nRates']
best_accuracies = [
    hidden_layer_df['accuracy'].max(),
    activation_df['accuracy'].max(),
    learning_rate_df['accuracy'].max()
]
colors_bar = ['#3498DB', '#27AE60', '#E74C3C']
bars = ax4.bar(experiments, best_accuracies, color=colors_bar, 
               edgecolor='black', linewidth=2, alpha=0.8)
ax4.set_ylabel('Best Test Accuracy', fontsize=12, fontweight='bold')
ax4.set_title('Best Accuracy by Experiment Type', fontsize=14, fontweight='bold')
ax4.set_ylim([min(best_accuracies) - 0.03, max(best_accuracies) + 0.03])

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + 0.005,
             f'{height:.4f}', ha='center', va='bottom', 
             fontsize=12, fontweight='bold')

# Add baseline reference line
ax4.axhline(y=baseline_accuracy, color='red', linestyle='--', 
            linewidth=2, label=f'Baseline: {baseline_accuracy:.4f}')
ax4.legend(fontsize=10, loc='lower right')
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../../outputs/figures/mlp_convergence_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print('\nVisualization saved!')
print('Location: outputs/figures/mlp_convergence_analysis.png')
print('=' * 60)

---

## 7. Hyperparameter Tuning with GridSearchCV

### What is GridSearchCV?

**GridSearchCV** performs exhaustive search over specified parameter combinations using cross-validation:

1. **Define parameter grid:** Specify values to test for each hyperparameter
2. **Generate combinations:** All possible combinations are created
3. **Cross-validation:** Each combination is evaluated using k-fold CV
4. **Select best:** Returns the combination with highest CV score

### Why use GridSearchCV?

| Benefit | Description |
|---------|-------------|
| **Systematic** | Tests all combinations, no guesswork |
| **Robust** | Cross-validation reduces overfitting to single split |
| **Automated** | No manual trial-and-error |
| **Reproducible** | Same grid always gives same results |

### Our Parameter Grid:

Based on our experiments, we focus on promising values:

| Parameter | Values | Rationale |
|-----------|--------|----------|
| `hidden_layer_sizes` | (100,50), (100,100), (150,100,50) | Top performers from Exp 1 |
| `activation` | 'relu', 'tanh' | Best from Exp 2 |
| `learning_rate_init` | 0.001, 0.01 | Optimal range from Exp 3 |
| `alpha` | 0.0001, 0.001 | L2 regularization strength |
| `batch_size` | 32, 64 | Mini-batch sizes |

**Total combinations:** 3 × 2 × 2 × 2 × 2 = **48**  
**With 5-fold CV:** 48 × 5 = **240 model trainings**

In [ ]:
# Define parameter grid based on experiment insights
param_grid = {
    'hidden_layer_sizes': [(100, 50), (100, 100), (150, 100, 50)],
    'activation': ['relu', 'tanh'],
    'learning_rate_init': [0.001, 0.01],
    'alpha': [0.0001, 0.001],
    'batch_size': [32, 64]
}

# Calculate total combinations
total_combinations = np.prod([len(v) for v in param_grid.values()])

print('=' * 60)
print('HYPERPARAMETER TUNING WITH GRIDSEARCHCV')
print('=' * 60)
print(f'\n Parameter Grid:')
for param, values in param_grid.items():
    print(f'   • {param}: {values}')
print(f'\n Total combinations: {total_combinations}')
print(f' With 5-fold CV: {total_combinations * 5} model trainings')
print(f'\n This may take several minutes...')
print('\n' + '-' * 60)
print('\n Starting GridSearchCV...\n')

# Create base MLP with early stopping
mlp_grid = MLPClassifier(
    random_state=42,
    max_iter=500,
    early_stopping=True,
    validation_fraction=0.1,
    n_iter_no_change=10
)

# Create GridSearchCV
grid_search = GridSearchCV(
    estimator=mlp_grid,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1,
    return_train_score=True
)

# Run grid search
start_time = time.time()
grid_search.fit(X_train, y_train)
grid_time = time.time() - start_time

print(f'\n Grid search completed in {grid_time:.2f} seconds ({grid_time/60:.1f} minutes)')
print('\n' + '=' * 60)
print('GRIDSEARCHCV RESULTS')
print('=' * 60)
print(f'\n Best Parameters:')
for param, value in grid_search.best_params_.items():
    print(f'   • {param}: {value}')
print(f'\n Best Cross-Validation Accuracy: {grid_search.best_score_:.4f} ({grid_search.best_score_*100:.2f}%)')
print('=' * 60)

In [ ]:
# Save and display top 10 configurations
results_df = pd.DataFrame(grid_search.cv_results_)
results_df = results_df.sort_values('rank_test_score')

# Select relevant columns
top_results = results_df[[
    'params', 'mean_test_score', 'std_test_score', 
    'mean_train_score', 'rank_test_score'
]].head(10)

print('=' * 60)
print('TOP 10 CONFIGURATIONS')
print('=' * 60)
print(top_results.to_string(index=False))

# Save to CSV
results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']].to_csv(
    '../../outputs/mlp_grid_search_results.csv', index=False
)
print('\n Full results saved to: outputs/mlp_grid_search_results.csv')
print('=' * 60)

---

## 8. Final Model Evaluation

### Comprehensive Evaluation Metrics

We evaluate the best model from GridSearchCV with multiple metrics:

| Metric | Description | Use Case |
|--------|-------------|----------|
| **Accuracy** | Overall correct predictions | Balanced classes |
| **Precision** | Correct positive predictions | Cost of false positives |
| **Recall** | Found positive instances | Cost of false negatives |
| **F1-Score** | Balance of P and R | Imbalanced classes |
| **Confusion Matrix** | Detailed error analysis | Understanding mistakes |

**Note:** ROC-AUC and Precision-Recall curves will be created by **Emircan (Member 3)** in **Task 2.17** for all models using One-vs-Rest approach.

In [ ]:
# Get best model and evaluate on test set
best_mlp = grid_search.best_estimator_
y_pred_best = best_mlp.predict(X_test)
best_accuracy = accuracy_score(y_test, y_pred_best)

print('=' * 60)
print('FINAL MODEL EVALUATION')
print('=' * 60)

print(f'\n Best Model Configuration:')
for param, value in grid_search.best_params_.items():
    print(f'   • {param}: {value}')

print(f'\n Performance Metrics:')
print(f'   • Cross-Validation Accuracy: {grid_search.best_score_:.4f}')
print(f'   • Test Set Accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)')
print(f'   • Baseline Accuracy: {baseline_accuracy:.4f}')
print(f'   • Improvement: {(best_accuracy - baseline_accuracy)*100:+.2f}%')

print(f'\nDetailed Classification Report:')
print('-' * 60)
print(classification_report(y_test, y_pred_best))
print('=' * 60)

print('\nNote: ROC-AUC, Precision-Recall curves and confusion matrices will be generated')
print('   by Emircan (Member 3) in Task 2.17 for comprehensive evaluation.')

---

## 9. Learning Curve Analysis

### What is a learning curve?

A learning curve shows how model performance changes with **training set size**:

- **Training score:** How well the model fits training data
- **Validation score:** How well the model generalizes

### Interpretation:

| Pattern | Diagnosis | Solution |
|---------|-----------|----------|
| Large gap, both improving | High variance (overfitting) | More data, regularization |
| Small gap, both low | High bias (underfitting) | More complex model |
| Curves converging | Good fit | Current model is appropriate |
| Training at 100% | Overfitting | Regularization, simpler model |

In [ ]:
# Generate learning curve
print('Generating learning curve (this may take a minute)...\n')

train_sizes, train_scores, val_scores = learning_curve(
    best_mlp, X_train, y_train,
    cv=5,
    n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 10),
    scoring='accuracy'
)

# Calculate mean and std
train_mean = train_scores.mean(axis=1)
train_std = train_scores.std(axis=1)
val_mean = val_scores.mean(axis=1)
val_std = val_scores.std(axis=1)

# Plot learning curve
plt.figure(figsize=(10, 6))
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, 
                 alpha=0.2, color='blue')
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, 
                 alpha=0.2, color='orange')
plt.plot(train_sizes, train_mean, 'o-', color='blue', linewidth=2, 
         markersize=8, label='Training Score')
plt.plot(train_sizes, val_mean, 'o-', color='orange', linewidth=2, 
         markersize=8, label='Validation Score')

plt.xlabel('Training Set Size', fontsize=12, fontweight='bold')
plt.ylabel('Accuracy', fontsize=12, fontweight='bold')
plt.title('Learning Curve - Best MLP Model', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../../outputs/figures/mlp_learning_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print('\n Learning curve saved!')
print(' Location: outputs/figures/mlp_learning_curve.png')

---

## 10. Save Model and Results

### Output Files:

| File | Location | Description |
|------|----------|-------------|
| `best_mlp_model.pkl` | models/ | Trained best MLP model |
| `mlp_grid_search_results.csv` | outputs/ | All GridSearchCV results |
| `mlp_experiment_results.json` | outputs/ | Complete experiment data |
| `mlp_convergence_analysis.png` | outputs/figures/ | Convergence plots |
| `mlp_learning_curve.png` | outputs/figures/ | Learning curve |

In [ ]:
# Save the best model
print('=' * 60)
print('SAVING MODEL AND RESULTS')
print('=' * 60)

# Save model
with open('../../models/best_mlp_model.pkl', 'wb') as f:
    pickle.dump(best_mlp, f)
print('\n Model saved: models/best_mlp_model.pkl')

# Prepare experiment results (excluding loss curves for JSON compatibility)
experiment_results = {
    'task': 'T2.5 - Neural Network (MLP Classifier)',
     'baseline': {
        'accuracy': float(baseline_accuracy),
        'iterations': int(baseline_mlp.n_iter_),
        'final_loss': float(baseline_mlp.loss_)
    },
    'experiment_1_hidden_layers': [
        {k: v for k, v in r.items() if k != 'loss_curve'} 
        for r in hidden_layer_results
    ],
    'experiment_2_activation': [
        {k: v for k, v in r.items() if k != 'loss_curve'} 
        for r in activation_results
    ],
    'experiment_3_learning_rate': [
        {k: v for k, v in r.items() if k != 'loss_curve'} 
        for r in learning_rate_results
    ],
    'best_model': {
        'parameters': {k: str(v) if isinstance(v, tuple) else v 
                      for k, v in grid_search.best_params_.items()},
        'cv_accuracy': float(grid_search.best_score_),
        'test_accuracy': float(best_accuracy),
        'improvement_over_baseline': float((best_accuracy - baseline_accuracy) * 100)
    },
    'note': 'ROC-AUC, PR curves and confusion matrices will be generated by Emircan (Member 3) in Task 2.16 and Task 2.17'
}

# Save as JSON
with open('../../outputs/mlp_experiment_results.json', 'w') as f:
    json.dump(experiment_results, f, indent=2)
print('Results saved: outputs/mlp_experiment_results.json')

print('\n' + '=' * 60)
print('ALL FILES SAVED SUCCESSFULLY')
print('=' * 60)

---

## 11. Summary and Conclusions

### Key Findings

#### Experiment 1: Hidden Layer Sizes
- Deeper architectures generally improved performance
- Diminishing returns after certain complexity
- Best architecture balanced capacity and generalization

#### Experiment 2: Activation Functions
- **ReLU:** Fast convergence, good performance
- **Tanh:** Stable training, competitive results
- **Logistic:** Slower convergence due to vanishing gradients

#### Experiment 3: Learning Rates
- **0.0001:** Too slow, many iterations needed
- **0.001:** Good balance (default works well)
- **0.01:** Faster convergence, still stable
- **0.1:** Risk of instability

### Convergence Behavior
- All models converged within max_iter
- Loss curves showed expected patterns
- Early stopping helped prevent overfitting

### Final Model Performance
- GridSearchCV found optimal hyperparameter combination
- Improvement achieved over baseline model
- Model ready for ensemble integration



In [ ]:
# Final summary
print('=' * 60)
print('TASK 2.5 COMPLETED SUCCESSFULLY')
print('=' * 60)
print(f'\n Results Summary:')
print(f'   • Baseline Accuracy: {baseline_accuracy:.4f} ({baseline_accuracy*100:.2f}%)')
print(f'   • Best Model Accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)')
print(f'   • Improvement: {(best_accuracy - baseline_accuracy)*100:+.2f}%')
print(f'\n Best Model Parameters:')
for param, value in grid_search.best_params_.items():
    print(f'   • {param}: {value}')
print(f'\n Generated Files:')
print('   • models/best_mlp_model.pkl')
print('   • outputs/mlp_grid_search_results.csv')
print('   • outputs/mlp_experiment_results.json')
print('   • outputs/figures/mlp_convergence_analysis.png')
print('   • outputs/figures/mlp_learning_curve.png')
print('\n' + '=' * 60)